In [2]:
import datetime as dt
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
import pandas as pd
from AE_Functions import *
from pandas import *
from io import BytesIO
import time
import sys
from forex_python.converter import CurrencyRates
from scipy.optimize import minimize
#import pyfolio
import numpy as np
#from nbconvert import *

Matplotlib is building the font cache; this may take a moment.


In [3]:
start=time.time()
def GeoMeanReturn(pct_change_column):
    return (((1 + pct_change_column).product()) ** (1 / (len(pct_change_column))) - 1) * 12

def ret(returns, weights):
    return returns.dot(weights)

def pf_var(covar,weights):
    return np.dot(weights,np.dot(weights,covar))

def pf_vol(covar,weights):
    return np.sqrt(np.dot(weights,np.dot(weights,covar)))

def get_ret_vol_sr(weights):
    weights = np.array(weights)
    #ret = np.sum(log_ret.mean() * weights) * 12
    ret = np.sum(GeoMeanReturn(log_ret) * weights)
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 12, weights)))
    sr = ret / vol
    return np.array([ret, vol, sr])


def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1


def constraint_check_sum(weights):
    return np.sum(weights) - 1

def constraint_vola(weights, Vola):
    return get_ret_vol_sr(weights)[1]

def minimize_volatility(weights):
    return get_ret_vol_sr(weights)[1]

In [7]:

df = pd.read_excel('Input.xlsx')

beg=(2010,1,1)
end=(2022,12,31)

ticker = (df['Ticker'].values)
col_names = (df['Name'].values)


In [9]:
col_names

array(['Lyxor MSCI World (LUX) UCITS ETF',
       'Vanguard FTSE Developed Europe UCITS ETF',
       'Deka iBoxx EUR Liquid Corps Diversified',
       'Deka iBoxx EUR Liquid Sovereign Diversified 5-7',
       'Xtrackers II ESG Global Aggregate Bond UCITS ETF'], dtype=object)

In [10]:
# -------------Tab Data---------------------------

from datetime import datetime 

quotes = pd.DataFrame()
tmp_min_dates = pd.DataFrame()
min_dates=[]

print('Daten werden heruntergeladen und aufbereitet....')

for i in ticker:
    try:

        quotes.insert(loc=len(quotes.columns), column=i, value=getQuote(i, beg, end, '1d'))
        tmp_min_dates=quotes
        min_dates.append(tmp_min_dates.index.min().to_pydatetime().date())
        quotes.dropna(inplace=True)
        # monthly_quotes=quotes.resample('M').agg(lambda x: x[-1])
        monthly_quotes_ForCCY = quotes.resample('BM').last()
    except Exception as err:
        print('Error: ', err)
        print('Security: ', i)
        sys.exit(0)


print('Daten verfügbar....')

Daten werden heruntergeladen und aufbereitet....
Error:  module 'yfinance' has no attribute 'pdr_override'
Security:  X010.DE


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
#-------------- Download Währungen --------------------------------
from currency_converter import CurrencyConverter
from datetime import timedelta
currencies = df['Currency'].drop_duplicates()
c = CurrencyConverter()
df_CCY = pd.DataFrame()
for ccy in currencies:
    fx_rates = []
    for day in monthly_quotes_ForCCY.index.date:
        try:
            x = c.convert(1,'EUR',ccy,day)
            fx_rates.append(x)
        except:
            dayminusone=day-timedelta(days=1)
            x = c.convert(1,'EUR',ccy,dayminusone)
            fx_rates.append(x)
            continue
    fx_rates_ = pd.Series(fx_rates, index=monthly_quotes_ForCCY.index.date)
    df_CCY.insert(loc=len(df_CCY.columns),column=ccy,value=fx_rates)

df_CCY.index=monthly_quotes_ForCCY.index.date

In [ ]:
# --------------Calculate Domestic Return ----------------------
monthly_quotes = pd.DataFrame()
monthly_quotes_data = monthly_quotes_ForCCY.join(df_CCY,how='outer')
for i in ticker:
    ccy = (df.loc[df['Ticker']==i,['Currency']].values)[0][0]
    monthly_quotes[str(i)] = monthly_quotes_data[i]/monthly_quotes_data[ccy]



In [ ]:
# ------------Tab Assets----------------

print('Worksheet Assets wird vorbereitet....')

#Performance = monthly_quotes.pct_change().dropna()
Performance = np.log(monthly_quotes/monthly_quotes.shift(1)).dropna()
#MeanReturn = (((1 + Performance).product()) ** (1 / (len(Performance) - 1)) - 1) * 12
MeanReturn = GeoMeanReturn(Performance)
vola = Performance.std() * np.sqrt(12)
cov = Performance.cov() * 12



list_ret = []
vol = []

for i in MeanReturn.values:
    list_ret.append(i)

for i in vola.values:
    vol.append(i)

weights = df['Wert in EUR/Sparrate'] / df['Wert in EUR/Sparrate'].sum()

df.insert(loc=6, column='Weights', value=weights)
df.insert(loc=7, column='Return', value=list_ret)
df.insert(loc=8, column='Standard Deviation', value=vol)
df.insert(loc=9, column='Sharpe Ratio [ohne Berücksichtigung von rf]',
        value=df['Return'] / df['Standard Deviation'])
df.insert(loc=10, column='Earliest Datapoint', value=min_dates)

df_MyPortfolio = monthly_quotes
df_MyPortfolio_returns = np.log(monthly_quotes/monthly_quotes.shift(1)).dropna()
#df_MyPortfolio_returns = df_MyPortfolio.pct_change().dropna()

#df_MyPortfolio.insert(loc=0,column=col_names, value=quotes)
#print(df_MyPortfolio_returns)

for i,w in enumerate(weights):
    df_MyPortfolio_returns.iloc[:,i] = df_MyPortfolio_returns.iloc[:,i] * w

    
df_MyPortfolio_returns.insert(loc=0,column='CompositeReturn',value=df_MyPortfolio_returns.sum(axis=1))


for i,w in enumerate(weights):
    df_MyPortfolio_returns.iloc[:,i+1] = df_MyPortfolio_returns.iloc[:,i+1] / w


df = df.fillna(0)

In [ ]:
##-------------Correlation Matrix------
print('Worksheet Correlation Matrix wird vorbereitet....')

df_corr = pd.DataFrame()
df_cov = pd.DataFrame()
dummy_df = pd.DataFrame()

for i in ticker:
    try:
        price = getQuote(i, beg, end, '1d')
        price = price.ffill()
        monthly_prices = price.resample('BM').last()  # filtert den letzten Tag im Monat // BM = Business Month // Alternative: 1ter Tag im Monat ('BMS').first() // BMS = Business Month Start
        #perf = monthly_prices.pct_change()
        perf = np.log(monthly_prices/monthly_prices.shift(1))
        df_corr.insert(loc=len(df_corr.columns), column=i, value=perf)
        df_cov.insert(loc=len(df_cov.columns), column=i, value=perf)
    except Exception as err:
        print('Error: ', err)
        print('Security: ', i)
        sys.exit(0)

df_corr.columns = col_names

df_corr.dropna(inplace=True)
df_corr = df_corr.corr()

In [ ]:
# -------------Tab Portfolio Statistics-------------
print('Worksheet Portfolio Statistics wird vorbereitet....')

##------------Expected Return----------


#Exp = df['Return'] * df['Weights']
#Exp = Exp.sum()
#Exp = ret(df['Return'].fillna(0),df['Weights'])
Exp = ret(df['Return'],df['Weights'])



##------------Standard Deviation-------

df_cov.columns = col_names
df_cov.dropna(inplace=True)
df_cov = df_cov.cov() * 12

#port_var = np.dot(df['Weights'].T, np.dot(df_cov, df['Weights']))
#port_vol = np.sqrt(port_var)

port_vol = pf_vol(df_cov,df['Weights'].T)
port_var = pf_var(df_cov,df['Weights'].T)

In [ ]:
# ---------- Efficient Frontier---------------------------
print('Efficient Frontier wird vorbereitet....')


log_ret = np.log(monthly_quotes/monthly_quotes.shift(1))
np.random.seed(42)
num_pf = 1000
all_weights = np.zeros((num_pf, len(monthly_quotes.columns)))
ret_arr = np.zeros(num_pf)
vol_arr = np.zeros(num_pf)
sharpe_arr = np.zeros(num_pf)
for pf in range(num_pf):
    weights = np.array(np.random.random(len(df)))
    weights = weights / np.sum(weights)
    all_weights[pf,:] = weights
    #ret_arr[pf] = np.sum(weights * 12 * log_ret.mean())
    ret_arr[pf] = np.sum(weights * GeoMeanReturn(log_ret))
    vol_arr[pf] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*12, weights)))
    sharpe_arr[pf] = ret_arr[pf] / vol_arr[pf]
    # pf_ExpRet.append(returns)
    # pf_std.append(volatility)
    # pf_sharpe.append(sharpe)
    # pf_weights.append(weights)

In [ ]:
##------------Optimize Portfolio------------------------
print('Portfolio-Optimierung gestartet .....')

Number_of_Opp_Sets = 6
opt_frontier_x_Soll = np.linspace(vol_arr.min(), vol_arr.max(), Number_of_Opp_Sets)
opt_frontier_x_Ist = []
opt_frontier_y = []
opt_frontier_weights = []
opt_frontier_SR = []

for Vola in opt_frontier_x_Soll:
    cons1 = ({'type':'eq','fun':constraint_check_sum},
            {'type':'eq','fun':lambda w:get_ret_vol_sr(w)[1] - Vola})
    bounds = ((0,1),)*len(df)
    init_guess = [df['Weights']]
    opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons1)
    opt_frontier_x_Ist.append(pf_vol(df_cov, opt_results.x))
    opt_frontier_weights.append(opt_results.x)
    opt_frontier_y.append(ret(df['Return'], opt_results.x))
    opt_frontier_SR.append(opt_results['fun']*(-1))

In [ ]:
#-------- MyPortfolio optimiert --------------------------------------------
print('Portfolio wird optimiert....')

Vola = port_vol
cons1 = ({'type': 'eq', 'fun': constraint_check_sum},
        {'type': 'eq', 'fun': lambda w: get_ret_vol_sr(w)[1] - Vola})
bounds = ((0, 1),) * len(df)
init_guess = [df['Weights']]
opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons1)
port_vol_opt = pf_vol(df_cov, opt_results.x)
port_weights_opt = opt_results.x
port_ExpRet_opt = ret(df['Return'], opt_results.x)
#port_SR_opt = opt_results['fun'] * (-1)
port_SR_opt = port_ExpRet_opt/port_vol_opt
port_var_opt = pf_var(df_cov,opt_results.x)

df.insert(loc=7,column='Weights [optimiert]',value=port_weights_opt)

#print('OptVola: ', port_vol_opt, 'OptExpRet: ', port_ExpRet_opt, 'OptSR: ',port_SR_opt, '\n')
#print('OptWeights', port_weights_opt)

In [ ]:
#----------- Max Sharpe Ratio Portfolio ------------------------------------

max_sr_ret = ret_arr[sharpe_arr.argmax()]
max_sr_vol = vol_arr[sharpe_arr.argmax()]
max_sr_weights = all_weights[sharpe_arr.argmax()]
max_sr_var = max_sr_vol**2
max_sr_SR = sharpe_arr.max()

In [ ]:
###----- Table MaxReturn for each Volatility Value --------------------------

df_tbl_MaxRet = pd.DataFrame(list(zip(opt_frontier_x_Ist, opt_frontier_y, opt_frontier_SR)),columns=['Volatility', 'Expected Return', 'Sharpe Ratio'])
df_frontier_weights = pd.DataFrame(opt_frontier_weights, columns=col_names)
#df_tbl_MaxRet = pd.concat([df_tbl_MaxRet, df_frontier_weights], sort=False, axis=1)
df_tbl_MaxRet_transposed = df_tbl_MaxRet.T
df_frontier_weights_transposed = df_frontier_weights.T

In [ ]:
# ----via XLSXwriter-----------

today = str(dt.date.today())
writer = pd.ExcelWriter('Output_' + today + '.xlsx', engine='xlsxwriter')

# -----set wb as variable------------

wb = writer.book

# -----formats as variables-------------

money_format = wb.add_format({'num_format': '#,##0.00', 'valign': 'center'})
percent_format_1 = wb.add_format({'num_format': '#0.00%', 'valign': 'center'})
percent_format_2 = wb.add_format({'bold':True,'num_format': '#0.00%'})

header_format_1 = wb.add_format({'bold': True, 'text_wrap': True, 'valign': 'center', 'border': 0})
header_format_2 = wb.add_format({'bold': True, 'text_wrap': True, 'valign': 'left', 'border': 0})
border_format = wb.add_format({'border': 1})

date_format_1 = wb.add_format({'num_format': 'dd/mm/yyyy'})
date_format_2 = wb.add_format({'num_format': 'd mmm yyyy'})
date_format_3 = wb.add_format({'num_format': 'dd.mm.yyyy'})

text_format_1 = wb.add_format({'valign': 'left'})
text_format_2 = wb.add_format({'valign': 'center'})

In [ ]:
# ----write DataFrames to sheets-------



df.to_excel(writer, sheet_name='Assets', float_format="%.6f", index=False, header=False, startrow=1)
dummy_df.to_excel(writer, sheet_name='Portfolio Statistics', float_format="%.6f", startcol=1, startrow=1,
                index=False, header=False)
df_corr.to_excel(writer, sheet_name='Correlation Matrix', float_format="%.6f", startcol=1, startrow=1,
                index=False, header=False)
monthly_quotes_data.to_excel(writer, sheet_name='Data', float_format="%.6f", index=False, header=False,
                        startrow=1, startcol=1)
df_tbl_MaxRet_transposed.to_excel(writer, sheet_name='Portfolio Statistics', float_format="%.3f", index=False, header=False,
                        startrow=35, startcol=5)
df_frontier_weights_transposed.to_excel(writer, sheet_name='Portfolio Statistics', float_format="%.3f", index=False, header=False,
                        startrow=41, startcol=5)

# ------assign worksheets to variables-------------

ws1 = writer.sheets['Assets']
ws2 = writer.sheets['Portfolio Statistics']
ws3 = writer.sheets['Data']
ws4 = writer.sheets['Correlation Matrix']

In [ ]:
# -----write headers & index-------------------

for col_num, value in enumerate(df.columns.values):
    ws1.write(0, col_num, value, header_format_1)

for col_num, value in enumerate(df_corr.columns.values):
    ws4.write(0, col_num + 1, value, text_format_2)

for row_num, value in enumerate(df_corr.columns.values):
    ws4.write(row_num + 1, 0, value, text_format_1)

col_names_inkl_CCY = col_names.tolist() + currencies.to_list()
for col_num, value in enumerate(col_names_inkl_CCY):
    ws3.write(0, col_num + 1, value, header_format_1)

for row_num, value in enumerate(monthly_quotes.index):
    ws3.write(row_num + 1, 0, value, date_format_1)

for row_num, value in enumerate(df_tbl_MaxRet.columns.values):
    ws2.write(row_num + 35, 4, value, header_format_2)
for row_num, value in enumerate(df_frontier_weights.columns.values):
    ws2.write(row_num + 41, 4, value, text_format_1)

for i in range(1,Number_of_Opp_Sets+1):
    ws2.write(34, 4+i,'Opportunity Set ' + str(i),header_format_1)

In [ ]:
# ------write Portfolio Statistics to Portfolio Sheet----------



ws2.set_comments_author('Andreas Eirich')
author = 'Andreas Eirich'

ws2.write('A1', 'PORTFOLIO [AKTUELL]:', header_format_2)

ws2.write('A3', 'Expected Return:')
ws2.write('B3', Exp, percent_format_1)
#ws2.write_comment('B3','ACHTUNG: Währungskomponente z.B. USD/EUR aktuell noch nicht berücksichtigt')

ws2.write('A4', 'Portfolio Variance:')
ws2.write('B4', port_var, percent_format_1)

ws2.write('A5', 'Portfolio Standard Deviation:')
ws2.write('B5', port_vol, percent_format_1)

ws2.write('A6', 'Sharpe Ratio [ohne Berücksichtigung von rf]:')
ws2.write('B6', Exp / port_vol, money_format)

ws2.write ('A8', 'PORTFOLIO [OPTIMIERT]:', header_format_2)

ws2.write('A10', 'Expected Return:')
ws2.write('B10', port_ExpRet_opt, percent_format_1)
ws2.write_comment('B10','DISCLAIMER: Historische Wertentwicklungen in der Vergangenheit sind kein verlässlicher Indikator für die künftige Wertentwicklung.',
                {'x_scale': 2})

ws2.write('A11', 'Portfolio Variance:')
ws2.write('B11', port_var_opt, percent_format_1)

ws2.write('A12', 'Portfolio Standard Deviation:')
ws2.write('B12', port_vol_opt, percent_format_1)

ws2.write('A13', 'Sharpe Ratio [ohne Berücksichtigung von rf]:')
ws2.write('B13', port_SR_opt, money_format)

ws2.write ('A15', 'PORTFOLIO [MAX SHARPE RATIO]:', header_format_2)

ws2.write('A17', 'Expected Return:')
ws2.write('B17', max_sr_ret, percent_format_1)

ws2.write('A18', 'Portfolio Variance:')
ws2.write('B18', max_sr_var, percent_format_1)

ws2.write('A19', 'Portfolio Standard Deviation:')
ws2.write('B19', max_sr_vol, percent_format_1)

ws2.write('A20', 'Sharpe Ratio [ohne Berücksichtigung von rf]:')
ws2.write('B20', max_sr_SR, money_format)

ws2.write('G40', '---------------------------------- PORTFOLIO GEWICHTUNGEN ----------------------------------', text_format_1)


In [ ]:
##------- Graph Efficient Frontier Linie -----------------

frontier_y = np.linspace(0.05,0.3,5) #todo Wert 20 erhöhen, vorher: 200


frontier_x = []
frontier_weights = []

for possible_return in frontier_y:
    cons2 = ({'type':'eq','fun':constraint_check_sum},
            {'type':'eq','fun':lambda w:get_ret_vol_sr(w)[0] - possible_return})
    result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons2)
    frontier_x.append(result['fun'])


#print('Vola: ', pf_vol(df_cov,w_min))
#print('ExpRet: ', ret(df['Return'],w_min))

In [ ]:
#--------------Plot Efficient Frontier--------------------


plt.figure(figsize=(15, 7))
plt.scatter(x=vol_arr, y=ret_arr, c=sharpe_arr, cmap='inferno') #for different color scales see: https://cran.r-project.org/web/packages/viridis/vignettes/intro-to-viridis.html
plt.colorbar(label='Sharpe ratio')
sns.set(style='darkgrid')
plt.title('Efficient Frontier')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.scatter(max_sr_vol, max_sr_ret, c='purple',s=50) #portfolio max Sharpe ratio
plt.annotate(text='Portfolio with highest SR',xy=(max_sr_vol,max_sr_ret),xytext=(7,0),textcoords='offset points',ha='left',va='center')
plt.plot(frontier_x,frontier_y,'r--',linewidth=3) #frontier line
plt.scatter(port_vol, Exp, c='red',s=50) # current portfolio
plt.annotate(text='Current Portfolio',xy=(port_vol,Exp),xytext=(7,0),textcoords='offset points',ha='left',va='center')
plt.scatter(port_vol_opt, port_ExpRet_opt, c='blue',s=50) # Optimized portfolio
plt.annotate(text='Optimized Portfolio',xy=(port_vol_opt,port_ExpRet_opt),xytext=(7,0),textcoords='offset points',ha='left',va='center')

imgdata2 = BytesIO()
plt.savefig(imgdata2, format="png", transparent=True)
imgdata2.seek(0)
ws2.insert_image(0, 2, "", {'image_data': imgdata2})

In [ ]:
# -----formatting--------------------

ws1.set_zoom(90)
ws2.set_zoom(85)
ws3.set_zoom(110)
ws4.set_zoom(110)

ws1.hide_gridlines(2)
ws2.hide_gridlines(2)
ws3.hide_gridlines(2)
ws4.hide_gridlines(2)

ws1.set_column('A:E', 25, text_format_1)
ws1.set_column('F:F', 40, money_format)
ws1.set_column('G:J', 25, percent_format_1)
ws1.set_column('K:M', 40, money_format)
ws1.set_column('B:B', 45, text_format_1)
ws1.set_column('L:L', 25, date_format_1)

ws2.set_column('A:A', 40)
ws2.set_column('B:B', 25)
ws2.set_column('C:C', 0.5)
ws2.set_column('D:AAA', 20, percent_format_1)
ws2.set_row(37,None,money_format)


ws3.set_column('B:AAA', 25, money_format)
ws3.set_column('A:A', 11)
ws3.freeze_panes(1, 0)

ws4.set_column('A:AAA', 25)

In [ ]:
# ------ Bar Chart Asset Sharpe Ratios-------------

length_df = str(len(df) + 1)

chart = wb.add_chart({'type': 'column'})

chart.add_series({
    'values': '=Assets!$I$2:$I$' + length_df,
    'categories': '=Assets!$E$2:$E$' + length_df,
    'fill': {'color': '#FF9900'}
})

chart.set_y_axis({'name': '=Assets!I1'})
chart.set_x_axis({'name': 'Assets'})
chart.set_legend({'none': True})
chart.set_size({'x_scale': 2, 'y_scale': 1.5})

ws1.insert_chart('O2', chart)

In [ ]:
# ---- Pie Chart Asset Allocation Current Portfolio-------
df_group = df[['Asset Class','Weights']]
df_group = df_group.groupby(df['Asset Class']).sum()
pie_plot = df_group.plot.pie(y='Weights',figsize=(5,5), autopct='%1.1f%%', legend=False, label='Current Portfolio')

#pie_plot.legend(loc='upper left')
imgdata5 = BytesIO()
pie_plot.figure.savefig(imgdata5,format='png',transparent=True)
imgdata5.seek(0)
ws2.insert_image('A25', "", {'image_data': imgdata5})



In [ ]:
# ---- Pie Chart Asset Allocation Optimized Portfolio-------

df_group = df[['Asset Class','Weights [optimiert]']]
df_group = df_group.groupby(df['Asset Class']).sum()
pie_plot = df_group.plot.pie(y='Weights [optimiert]',figsize=(5,5), autopct='%1.1f%%', legend=False, label='Optimized Portfolio')

#pie_plot.legend(loc='upper left')
imgdata6 = BytesIO()
pie_plot.figure.savefig(imgdata6,format='png',transparent=True)
imgdata6.seek(0)
ws2.insert_image('A47', "", {'image_data': imgdata6})

In [ ]:
#-----------Graph Historic Returns--------------------------

plt.figure(figsize=(12,len(col_names)*2))
# for c, i in enumerate(quotes.columns.values):
#     plt.plot(quotes[i],label=col_names[c])
#quotes.insert(loc=0,column='Composite Prices',value=df_MyPortfolio)
quotes.columns=col_names
quotes.plot(title="Historic Prices", figsize=(12,len(col_names)*2),subplots=True)
#plt.title('Historic prices')
plt.xlabel('Date',fontsize=13)
plt.ylabel('Prices',fontsize=13)
plt.legend(loc='upper left', borderaxespad=1)
imgdata1 = BytesIO()
plt.savefig(imgdata1, format="png", transparent=True)
imgdata1.seek(0)
ws1.insert_image('N25', "", {'image_data': imgdata1})

#df_MyPortfolio = df_MyPortfolio.cumsum()
#df_MyPortfolio.plot(x='CompositeReturn',title="Strategies Returns", figsize=(12,10),subplots=True)


In [ ]:
#-------------------Performance Measurement -----------------------------


pyfolio.tears.create_returns_tear_sheet(pd.Series(df_MyPortfolio_returns['CompositeReturn']))
#pyfolio.create_full_tear_sheet(pd.Series(df_MyPortfolio_returns['CompositeReturn']))
#plt.show()


imgdata3 = BytesIO()
plt.savefig(imgdata3, format="png", transparent=True)
imgdata3.seek(0)
ws2.insert_image(0, 15, "", {'image_data': imgdata3})


col_names = np.insert(col_names,0,'CompositeReturn',axis=0)
df_MyPortfolio_returns.columns=col_names
df_MyPortfolio_returns.plot(kind='hist',bins=15,subplots=True,figsize=(16,10))

imgdata4 = BytesIO()
plt.savefig(imgdata4, format="png", transparent=True)
imgdata4.seek(0)
ws1.insert_image(0, 40, "", {'image_data': imgdata4})



writer.save()
writer.close()
print('Vorgang abgeschlossen')
ende=time.time()
print('Dauer Programmablauf: ','{:5.3f} min'.format((ende-start)/60),end=' ')
